# Search Engine with Tools and Agents 

In [11]:
import os 

In [40]:
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [28]:
## Arxiv--Research 

## Tools creation 

from langchain_community.tools import ArxivQueryRun , WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [29]:
## Used the inbuilt tool of wikipedia 

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 1 , doc_content_chars_max = 250) 
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)
wiki.name

'wikipedia'

In [30]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results = 5 , doc_content_chars_max = 250)
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv)
arxiv.name

'arxiv'

In [31]:
tools = [wiki , arxiv]


In [32]:
## Making our custom tools [Rag tool] 

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings 
from langchain_text_splitters import RecursiveCharacterTextSplitter 


In [34]:
loader = WebBaseLoader("https://cookbook.openai.com/examples/how_to_build_a_tool-using_agent_with_langchain")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C284873F10>, search_kwargs={})

In [37]:
from langchain.tools.retriever import create_retriever_tool 
retriever_tool = create_retriever_tool(retriever , "Langchain-Search" , "Search any information about how tools can be made using Langcahin")
retriever_tool.name

'Langchain-Search'

In [38]:
tools = [arxiv , wiki , retriever_tool]

In [39]:
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=5, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'f:\\anaconda3\\envs\\lc-academy-env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langchain-Search', description='Search any information about how tools can be made using Langcahin', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001C2D64F8220>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS objec

In [49]:
# Run all the tools with agents and LLM Models 

# Tools , LLM --> AgentExecutor 
import openai
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key = groq_api_key , model_name = "Llama3-8b-8192")

In [50]:
## Prompt Template 
from langchain import hub 
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [51]:
## Agents 

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [52]:
## Agent Executer 

from langchain.agents import AgentExecutor 

agent_executor = AgentExecutor(agent = agent,tools = tools , verbose = True)

In [53]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [54]:
agent_executor.invoke({"input": "Tell me about how to create a tools using langchain and openai"})




> Entering new AgentExecutor chain...

Invoking: `Langchain-Search` with `{'query': 'How to create a tool using Langchain and OpenAI'}`


How to build a tool-using agent with LangChainTopicsAboutAPI DocsSourceToggle themeToggle themeMay 2, 2023How to build a tool-using agent with LangChainColin Jarvis(OpenAI)Open in GithubView as MarkdownThis notebook takes you through how to use LangChain to augment an OpenAI model with access to external tools. In particular, you'll be able to create LLM agents that use custom tools to answer user queries.
What is Langchain?
LangChain is a framework for developing applications powered by language models. Their framework enables you to build layered LLM-powered applications that are context-aware and able to interact dynamically with their environment as agents, leading to simplified code for you and a more dynamic user experience for your customers.
Why do LLMs need to use Tools?

# Langchain imports
from langchain.agents import Tool, AgentExecutor

{'input': 'Tell me about how to create a tools using langchain and openai',
 'output': 'Based on the results of the previous tool calls, it appears that you are interested in learning how to create a tool using Langchain and OpenAI. The second tool call yielded information on a research paper that discusses the use of Langchain for generating breast ultrasound reports. This suggests that Langchain has the potential to be used in various applications, including healthcare.\n\nTo answer your original question, creating a tool using Langchain and OpenAI involves several steps. First, you need to import the necessary libraries and set up a connection to a Pinecone vector database. Then, you can build an LLM agent that leverages a modified version of the ReAct framework to do chain-of-thought reasoning. You can also provide the LLM with access to previous steps in the conversation and create a knowledge base of relevant information.\n\nHere is an example of how you can create a simple tool 

In [55]:
agent_executor.invoke({"input": "what is GAN's"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Generative Adversarial Networks'}`


Page: Generative adversarial network
Summary: A generative adversarial network (GAN) is a class of machine learning frameworks and a prominent framework for approaching generative artificial intelligence. The concept was initially developed by Ian Go
Invoking: `arxiv` with `{'query': 'Generative Adversarial Networks'}`


Published: 2018-10-30
Title: Improved Network Robustness with Adversary Critic
Authors: Alexander Matyasko, Lap-Pui Chau
Summary: Ideally, what confuses neural network should be confusing to humans. However,
recent experiments have shown that small, 
Invoking: `Langchain-Search` with `{'query': 'what is generative adversarial network used for'}`
responded: Based on the information provided, it appears that the user is interested in learning more about Generative Adversarial Networks (GANs). To provide a more comprehensive answer, I will use the tools to 

{'input': "what is GAN's",
 'output': "Thank you for providing the results of the previous tool calls. Based on the outputs, it seems that LangChain is a framework for developing applications powered by language models, and it allows LLMs to interact dynamically with their environment using tools.\n\nConsidering the conversation about Generative Adversarial Networks (GANs), I'd like to provide a direct response. GANs are a type of deep learning framework that uses a combination of a generator and discriminator to generate new, synthetic data that resembles existing data. This technique has been applied to various domains, including computer vision, natural language processing, and audio processing.\n\nIn the context of GANs, tools like Wikipedia and ArXiv can be used to gather information about the concept, its applications, and its limitations. Additionally, LangChain can be used to augment an LLM-powered application with access to external tools, enabling the model to interact dynami